## Re-process ATAC-seq data for TE detection

**3/23/20**

Previously processed with ENCODE pipeline, which used Bowtie2 to align and removed multimapped reads. This would eliminate most of the TE signal, which may explain why few peaks called from the ENCODE-processed data overlap with our R499 3'UTR TEs of interest.

*We also don't know if removing blacklist regions is a good idea if goal is to detect repetitive TEs. Gotta figure this out.*

________________________________________________________________

### Merge fastq technical reps

Conditions: B16_cas, B16_SKO, R499_cas, R499_SKO <br>
5 biological replicates each <br>
3 sequencing runs each (FGC1587, FGC1609, FGC1748)

fastq files stored at: <br>
/home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/fastq/merged

In [ ]:
from subprocess import call
import os
import sys

names = ["FGC1587", "FGC1609", "FGC1748"]

conditions = ["B16_cas", "B16_SKO", "R499_cas", "R499_SKO"]

for i in range(0, len(conditions)):
    for j in range(1, 6):
        id = conditions[i] + "_" + str(j)

        for k in range(1, 3):

            fastq1 = names[0] + "_" + id + ".R" + str(k) + ".fastq.gz"
            fastq2 = names[1] + "_" + id + ".R" + str(k) + ".fastq.gz"
            fastq3 = names[2] + "_" + id + ".R" + str(k) + ".fastq.gz"

            out_fastq = "merged/" + id + ".R" + str(k) + ".fastq.gz"

            command = "cat " + fastq1 + " " + fastq2 + " " + fastq3 + " > " + out_fastq

            os.system(command)

### Trim adapter sequences

cutadapt version 2.9

ATAC adapters:
R1 - CTGTCTCTTATACACATCTCCGAGCCCACGAGAC
R2 - CTGTCTCTTATACACATCTGACGCTGCCGACGA

In [ ]:
### Install cutadapt ###
module load python/3.6.3
python3 -m pip install --user --upgrade cutadapt

python /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/scripts/mRFAR_v7/v1/cutadapt_jobScripts.py
# python ~/Dropbox/Minn/B16_R499_Stat1KO_ATAC/processing_scripts/cutadapt_jobScripts.py

### Align with Bowtie2

bowtie2 version 2.3.4.1

Bowtie2 by default searches for multiple alignments, reports best one. Use standard parameter settings [--very-sensitive, --end-to-end, -p 16]. Output as sorted bam file.

**For TE detection:** <br>
Use MAPQ score (>5) to filter for "unique"/"reliably mapped" reads. <br>
Higher mapping quality = more "unique". MAPQ indicates the degree of confidence that a read is from the aligned point of origin. *The larger the gap between the best alignment score and the second-best alignment score, the higher its mapping quality will be, and the more unique the alignment is.* MAPQ=10 --> at least a 1 in 10 chance that read truly originated elsewhere. 

Raw bam files located at: <br>
/home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/bam

**Alignment summaries:**
- /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/bam/bowtie2_logs/error*
- Overall mapping rate: 99%
- Across all samples, 40-45% reads were concordantly mapped 1x. 
- 50-55% reads were concordantly multi-mapped - these must be removed for TE analysis with MAPQ threshold filter**
__________________________________________

Need to figure out how squire handles this. He et al. 2019 assigned multimapped reads to only one region. If we do this then cannot analyze individual TE loci.

In [ ]:
module load bowtie2/2.3.4.1

python /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/scripts/mRFAR_v7/v1/bowtie2_TE_jobScripts.py
# python ~/Dropbox/Minn/B16_R499_Stat1KO_ATAC/processing_scripts/bowtie2_TE_jobScripts.py

mv output_logs/bowtie2/*.err /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/bam/bowtie2_logs/

cd /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/bam/bowtie2_logs/
for file in *;  do echo $file;  grep "overall" $file; done
for file in *;  do echo $file;  grep "concordantly exactly 1" $file; done
for file in *;  do echo $file;  grep "concordantly >1 times" $file; done

In [ ]:
# Number of mapped reads #

# for file in *.bam;  do samtools view -c $file; done
for file in *.bam;  do echo $file;  samtools idxstats $file | awk '{s+=$3+$4} END {print s}'; done

# Check flagstats
for file in *.bam;  do samtools flagstat $file; done

### Filter bam files

#### Retain only properly paired, unique reads

Remove: <br>
1. unmapped reads (-F 1804) <br>
2. reads that are not properly paired (-f 2) <br>
3. likely multi-mapped reads (-q 5)

In [ ]:
# -f 2 Only output alignments "mapped in proper pairs"
# -F 1804 Do not output any alignments that are "unmapped", "mate unmapped", "not primary alignment", "read fails platform/vendor quality checks", "read is PCR or optical duplicate"
# -q 5 Filter out reads with MAPQ < 5

cd /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/bam
for file in *bam;  do echo $file;  samtools view -h -F 1804 -f 2 -q 5 -b $file > filtered/$file; done

In [ ]:
### Filter RNA bam files ###

cd /home/jingyaq/Minn/data/B16_R499_Stat1KO_RNA/squire/aligned
for dir in *;  do echo $dir;  samtools view -h -F 1804 -f 2 -q 5 -b $dir/$dir.bam > filtered/$dir.bam; done

#### Mark and remove PCR duplicates

In [ ]:
module load java/openjdk-1.8.0

cd /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/bam/filtered/
python /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/scripts/markDuplicates_jobScripts.py
rm *.out *.err

#### Remove mitochondrial reads

**Check if name is "chrM" or "MT"!!**

In [ ]:
python /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/scripts/remove_chrM_jobScripts.py MT
rm *.out *.err

# Index bam files #
python /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/scripts/bamIndex_jobScripts.py

### tn5 shift

In [ ]:
export PATH="/home/jingyaq/anaconda2/bin:$PATH"
python /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/scripts/tn5shift_jobScripts.py
rm *.err *.out

# Sort and index output bam files #
cd /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/bam/filtered/deduped/noChrM/tn5shift
python /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/scripts/sort_bam.py 
python /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/scripts/bamIndex_jobScripts.py

rm output* error* *_tn5.bam

### Convert to bigwigs

In [ ]:
export PATH="/home/jingyaq/anaconda2/bin:$PATH"
python /home/jingyaq//Minn/data/B16_R499_Stat1KO_ATAC/scripts/bam_to_bigwig_jobScripts.py
rm output* error*

### Merge bam files

In [ ]:
cd /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/bam/filtered/deduped/noChrM/tn5shift

bsub -M 140000 -o B16_cas_merge.out -e B16_cas_merge.err samtools merge merged/B16_cas.sorted_dedup_noChrM_tn5_sorted.bam B16_cas_*.bam

bsub -M 140000 -o B16_SKO_merge.out -e B16_SKO_merge.err samtools merge merged/B16_SKO.sorted_dedup_noChrM_tn5_sorted.bam B16_SKO_*.bam

bsub -M 140000 -o R499_cas_merge.out -e R499_cas_merge.err samtools merge merged/R499_cas.sorted_dedup_noChrM_tn5_sorted.bam R499_cas_*.bam

bsub -M 140000 -o R499_SKO_merge.out -e R499_SKO_merge.err samtools merge merged/R499_SKO.sorted_dedup_noChrM_tn5_sorted.bam R499_SKO_*.bam

rm *.out *.err

# Index bam files
python /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/scripts/bamIndex_jobScripts.py

# Convert to bigwig
python /home/jingyaq//Minn/data/B16_R499_Stat1KO_ATAC/scripts/bam_to_bigwig_jobScripts.py

In [ ]:
### Merge RNA bam files ###

cd /home/jingyaq/Minn/data/B16_R499_Stat1KO_RNA/squire/aligned/filtered

bsub -M 140000 -o B16_cas_merge.out -e B16_cas_merge.err samtools merge merged/B16_cas_sorted.bam B16_cas_*.bam

bsub -M 140000 -o B16_SKO_merge.out -e B16_SKO_merge.err samtools merge merged/B16_SKO_sorted.bam B16_SKO_*.bam

bsub -M 140000 -o R499_cas_merge.out -e R499_cas_merge.err samtools merge merged/R499_cas_sorted.bam R499_cas_*.bam

bsub -M 140000 -o R499_SKO_merge.out -e R499_SKO_merge.err samtools merge merged/R499_SKO_sorted.bam R499_SKO_*.bam

rm *.out *.err

# Index bam files
python /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/scripts/bamIndex_jobScripts.py

# Convert to bigwig
export PATH="/home/jingyaq/anaconda2/bin:$PATH"
python /home/jingyaq//Minn/data/B16_R499_Stat1KO_ATAC/scripts/bam_to_bigwig_jobScripts.py

### Final read counts

### Plot TE coverage with unique read bam files

In [ ]:
### Deeptools to generate coverage profiles ###

# export PATH="/home/jingyaq/anaconda2/bin:$PATH"
# version 3.1.3

# TE.DE_regions_byLocus_R499UP.bed
# TE.DE_regions_byLocus_R499DOWN.bed
# TE.DE_regions_byLocus_R499UP_3UTR.bed
# TE.DE_regions_byLocus_R499DOWN_3UTR.bed
# TE_regions_byLocus_RANDOM1.bed
# TE_regions_byLocus_RANDOM2.bed

computeMatrix reference-point -S /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/bigwig/tn5/v1/B16_cas.R1.trim_pooled.PE2SE.nodup_noChrM_tn5_sorted_merged.bw /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/bigwig/tn5/v1/B16_SKO.R1.trim_pooled.PE2SE.nodup_noChrM_tn5_sorted_merged.bw /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/bigwig/tn5/v1/R499_cas.R1.trim_pooled.PE2SE.nodup_noChrM_tn5_sorted_merged.bw /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/bigwig/tn5/v1/R499_SKO.R1.trim_pooled.PE2SE.nodup_noChrM_tn5_sorted_merged.bw -R /home/jingyaq/Minn/data/B16_R499_Stat1KO_RNA/squire/TE_DE_regions/TE.DE_regions_byLocus_R499UP.bed --referencePoint center --upstream 2500 --downstream 2500 --binSize 1 --skipZeros -p 12 -o  /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/mRFAR_v7/deeptools/squire_TE/computeMatrix/TE.DE_regions_byLocus_R499UP_computeMatrix.gz

In [ ]:
python /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/scripts/mRFAR_v7/v1/computeMatrix_TE_DE_jobScripts.py
# python /Users/jingyaqiu/Dropbox/Minn/B16_R499_Stat1KO_ATAC/analysis_scripts/computeMatrix_TE_DE_jobScripts.py

#### Plot heatmaps/profiles

In [ ]:
plotHeatmap -m /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/mRFAR_v7/deeptools/squire_TE/computeMatrix/TE.DE_regions_byLocus_R499DOWN_3UTR_computeMatrix.gz -out /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/mRFAR_v7/deeptools/squire_TE/hm/TE.DE_regions_byLocus_R499DOWN_3UTR_heatmap.pdf --colorMap RdBu_r --regionsLabel "R499DOWN 3' UTR TE loci" --samplesLabel "B16 cas" "B16 SKO" "R499 cas" "R499 SKO" --heatmapHeight 14 --refPointLabel "peak center" --xAxisLabel "(bp)" --legendLocation "none" --plotFileFormat pdf --zMax 16

plotHeatmap -m /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/mRFAR_v7/deeptools/squire_TE/computeMatrix/TE.DE_regions_byLocus_R499UP_3UTR_computeMatrix.gz -out /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/mRFAR_v7/deeptools/squire_TE/hm/TE.DE_regions_byLocus_R499UP_3UTR_heatmap.pdf --colorMap RdBu_r --regionsLabel "R499UP 3' UTR TE loci" --samplesLabel "B16 cas" "B16 SKO" "R499 cas" "R499 SKO" --heatmapHeight 14 --refPointLabel "peak center" --xAxisLabel "(bp)" --legendLocation "none" --plotFileFormat pdf --zMax 16

plotHeatmap -m /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/mRFAR_v7/deeptools/squire_TE/computeMatrix/TE.DE_regions_byLocus_R499DOWN_computeMatrix.gz -out /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/mRFAR_v7/deeptools/squire_TE/hm/TE.DE_regions_byLocus_R499DOWN_heatmap.pdf --colorMap RdBu_r --regionsLabel "R499DOWN TE loci" --samplesLabel "B16 cas" "B16 SKO" "R499 cas" "R499 SKO" --heatmapHeight 14 --refPointLabel "peak center" --xAxisLabel "(bp)" --legendLocation "none" --plotFileFormat pdf --zMax 16

plotHeatmap -m /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/mRFAR_v7/deeptools/squire_TE/computeMatrix/TE.DE_regions_byLocus_R499UP_computeMatrix.gz -out /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/mRFAR_v7/deeptools/squire_TE/hm/TE.DE_regions_byLocus_R499UP_heatmap.pdf --colorMap RdBu_r --regionsLabel "R499UP TE loci" --samplesLabel "B16 cas" "B16 SKO" "R499 cas" "R499 SKO" --heatmapHeight 14 --refPointLabel "peak center" --xAxisLabel "(bp)" --legendLocation "none" --plotFileFormat pdf --zMax 16

plotHeatmap -m /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/mRFAR_v7/deeptools/squire_TE/computeMatrix/TE_regions_byLocus_RANDOM1_computeMatrix.gz -out /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/mRFAR_v7/deeptools/squire_TE/hm/TE_regions_byLocus_RANDOM1_heatmap.pdf --colorMap RdBu_r --regionsLabel "RANDOM1 TE loci" --samplesLabel "B16 cas" "B16 SKO" "R499 cas" "R499 SKO" --heatmapHeight 14 --refPointLabel "peak center" --xAxisLabel "(bp)" --legendLocation "none" --plotFileFormat pdf --zMax 16

plotHeatmap -m /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/mRFAR_v7/deeptools/squire_TE/computeMatrix/TE_regions_byLocus_RANDOM2_computeMatrix.gz -out /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/ENCODE_processed/mRFAR_v7/deeptools/squire_TE/hm/TE_regions_byLocus_RANDOM2_heatmap.pdf --colorMap RdBu_r --regionsLabel "RANDOM2 TE loci" --samplesLabel "B16 cas" "B16 SKO" "R499 cas" "R499 SKO" --heatmapHeight 14 --refPointLabel "peak center" --xAxisLabel "(bp)" --legendLocation "none" --plotFileFormat pdf --zMax 16

### Peak calling

In [ ]:
export PATH="/home/jingyaq/anaconda2/bin:$PATH"

python /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/scripts/mRFAR_v7/v1/macs2_TE_jobScripts.py
python /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/scripts/mRFAR_v7/v1/macs2_TE_merged_jobScripts.py

In [ ]:
cd /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/peaks
for dir in *;  do cp $dir/*.narrowPeak narrowPeaks; done
for dir in merged/*;  do cp $dir/*.narrowPeak narrowPeaks/; done

# R #
cd /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/peaks/narrowPeaks

# Add "chr" to chromosome names
files <- list.files("/home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/peaks/narrowPeaks")
lapply(1:length(files), function(x) {
    print(files[x])
    df <- read.table(files[x], sep="\t", stringsAsFactors=F, header=F)
    df$V1 <- paste0("chr", df$V1)
    write.table(df, file=files[x], sep="\t", row.names=F, col.names=F, quote=F)
})

### Blacklist filter

Might be optimal to filter blacklist reads before peak calling (I think ENCODE recommends this). But seems like it isn't a huge improvement over simply filtering out peaks in blacklist regions. https://www.biostars.org/p/184537/

In [ ]:
cd /home/jingyaq/Minn/data/epi_ATAC/ATAC/peaks/narrowPeaks

for file in *.narrowPeak;  do echo $file;  bedtools intersect -v -a $file -b /home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/resources/mm10.blacklist.bed > filt/$file.filt; done

### Make consensus peakset

1. Replicate - peak present in both replicates (minOverlap=2)
2. IDR - reproducible peaks identified by IDR statistical procedure

Combine replicate or IDR peaks to get non-overlapping set for each condition. Write out original peakset and fixed-width peakset. ChromVAR reduces peaks to non-overlapping set based on which peak has stronger signal [did not use chromVAR!]

#### Use replicate peaks

In [ ]:
rm(list=ls())

library(DiffBind)
library(rtracklayer)
library(GenomicRanges)
library(SummarizedExperiment)

dir.path <- "/home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/"

conditions <- c("B16_cas", "B16_SKO", "R499_cas", "R499_SKO")

bamfiles <- list.files("/home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/bam/filtered/deduped/noChrM/tn5shift", pattern="bam$", full.names=F)
sampleID <- sapply(strsplit(bamfiles, split="\\."), function(x) x[[1]])
Condition <- sapply(strsplit(sampleID, split="_"), function(x) paste(x[1:2], collapse="_"))
Replicate <- rep(1:5, 4)
                    
bamReads <- paste0("/home/jingyaq/Minn/data/B16_R499_Stat1KO_ATAC/data/bam/filtered/deduped/noChrM/tn5shift/", bamfiles)
Peaks <- unlist(lapply(sampleID, function(x) paste0(dir.path, "data/peaks/narrowPeaks/filt/", x, "_peaks.narrowPeak.filt")))
PeakCaller <- rep("narrow", length(sampleID))
ss <- data.frame(sampleID, Condition, Replicate, bamReads, Peaks, PeakCaller)
                       
# Load in individual sample sheet
db <- dba(sampleSheet=ss)

# Get replicate peaks based on naive overlaps (n=2), condition-specific
db.replicate_peaks <- dba.peakset(db, consensus=DBA_CONDITION, minOverlap=2)
db.consensus <- dba(db.replicate_peaks, mask=db.replicate_peaks$masks$Consensus, minOverlap=1)

# Write out replicate peaks for each condition
lapply(1:length(conditions), function(x) {
    print(conditions[x])
    idx <- which(names(db.consensus$masks) == conditions[x])
    db.cond <- dba(db.consensus, mask=db.consensus$masks[[idx]], minOverlap=1)
    rep_peaks <- dba.peakset(db.cond, bRetrieve=TRUE)
    write.table(data.frame(rep_peaks)[,1:3], file=paste0(dir.path, "data/peaks/consensus/", conditions[x], "_replicate_peaks.bed"), quote=F, sep="\t", col.names=F, row.names=F)
})

# Consensus peakset (combine replicate peaks from each condition)
consensus_peaks <- dba.peakset(db.consensus, bRetrieve=TRUE)
write.table(data.frame(consensus_peaks)[,1:3], file=paste0(dir.path, "data/peaks/consensus/consensus_replicate_peaks.bed"), quote=F, sep="\t", col.names=F, row.names=F)
saveRDS(consensus_peaks, paste0(dir.path, "data/peaks/consensus/consensus_replicate_peaks.rds"))

# Fixed-width peaks
consensus_peaks_fixed <- resize(consensus_peaks, width=750, fix="center")
write.table(data.frame(consensus_peaks_fixed)[,1:3], file=paste0(dir.path, "data/peaks/consensus/consensus_replicate_peaks_fixed750.bed"), quote=F, sep="\t", col.names=F, row.names=F)
saveRDS(consensus_peaks_fixed, paste0(dir.path, "data/peaks/consensus/consensus_replicate_peaks_fixed750.rds"))